In [1]:
#%pip install -q wandb
import wandb
wandb.login()

#from google.colab import drive
#drive.mount('/content/drive')

wandb: Currently logged in as: woodleighj (jackwoodleigh). Use `wandb login --relogin` to force relogin


True

In [2]:
#%pip install -q einops
import os
import torch
import torchvision
import torchvision.transforms as transforms
from Diffusion import DiffusionModel
from utils import *

In [3]:
img_size = 32
batch_size = 64
epochs = 200
learning_rate = 0.0002
min_learning_rate= 0.00001
log = True
noise_steps = 1000
noise_schedule = "linear"
block_structure = [1, 1, 1]
block_multiplier = [2, 2, 2]
d_model = 64
ema_weight = 0.99975

In [4]:
transform = transforms.Compose([
            transforms.Resize(32),
            #transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # normalize mean and std for n=3 color channels
        ])

In [5]:
training_set = torchvision.datasets.CIFAR10('./data', train=True, transform=transform, download=True)
testing_set = torchvision.datasets.CIFAR10('./data', train=False, transform=transform, download=True)
training_set, validation_set = torch.utils.data.random_split(training_set, [45000, 5000])

Files already downloaded and verified
Files already downloaded and verified


In [6]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True, num_workers=10,  pin_memory=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, shuffle=True, num_workers=10, pin_memory=True)
testing_loader = torch.utils.data.DataLoader(testing_set, batch_size=batch_size, shuffle=True, num_workers=10, pin_memory=True)

In [7]:
model = DiffusionModel(
    d_model=d_model,
    block_structure=block_structure,
    block_multiplier=block_multiplier,
    learning_rate=learning_rate,
    min_learning_rate=min_learning_rate,
    in_channels=3,
    out_channels=3,
    noise_steps=noise_steps,
    beta_start=1e-4,
    beta_end=0.02,
    img_size=img_size,
    noise_schedule=noise_schedule,
    ema_weight=ema_weight,
    device="cuda"
)

In [8]:
model.print_parameter_count()

47537795


In [9]:
if log:
    wandb.init(
        project="DenoiseDiffusion",
        config={
        "dataset": "CIFAR-100",
        "img_size": img_size,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epochs,
        "noise_schedule": noise_schedule,
        "d_model": d_model,
        "block_structure": block_structure,
        "block_multiplier": block_multiplier,
        "EMA": ema_weight,
        "Type": "DDPM_V2",
        "Save": "Save-0"
        }
    )
    run_name = wandb.run.name
    save_file_name = f"model_save_{run_name}.pth"
else:
    save_file_name = "model_save.pth"

path = "./saves/" + save_file_name

model.train_model(training_loader, validation_loader, epochs, log=log, save_path=path)
wandb.finish()

Epoch 0...


  0%|                                                                                          | 0/704 [00:00<?, ?it/s]C:\Users\Jack\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:23<00:00,  3.36it/s]


Training Loss: 0.2602018557478335, Validation Loss: 0.053422176338072065
Model Saved.
Epoch 1...


100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:23<00:00,  3.37it/s]


Training Loss: 0.04989123821459626, Validation Loss: 0.0417616244735597
Model Saved.
Epoch 2...


...

100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:24<00:00,  3.19it/s]


Training Loss: 0.028997574829686942, Validation Loss: 0.027875356340804434
Model Saved.
Epoch 199...


100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:24<00:00,  3.18it/s]


Training Loss: 0.029068660278095525, Validation Loss: 0.028624365683880788
Model Saved.


Training_Loss,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation_Loss,█▄▃▃▂▂▂▂▂▂▂▁▃▂▂▁▂▂▁▂▁▂▁▂▂▁▂▁▂▂▂▁▂▂▁▁▂▂▂▂
Training_Loss,0.02907
Validation_Loss,0.02862
